In [88]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import sys
sys.path.append('../src')

## Visualizations

### Univariate Analysis

#### 1. Activity groups related to shark attacks

In [89]:
from utils import clean_activity, assign_activity_group

# 1. Load data
df = pd.read_excel('../data/raw/GSAF5.xls')

# 2. Clean and categorize activity
df = clean_activity(df)
df = assign_activity_group(df)

# 3. Count number of attacks per activity group
activity_counts = df['Activity_group'].value_counts().reset_index()
activity_counts.columns = ['Activity Group', 'Number of Attacks']

# 4. Label as primary / other
primary_activities = ['Swimming', 'Surfing', 'Diving', 'Wading']
activity_counts['Activity Type'] = activity_counts['Activity Group'].apply(
    lambda x: 'Primary' if x in primary_activities else 'Other'
)

# 5. Define colors
custom_colors = {'Primary': 'steelblue', 'Other': 'lightcoral'}

# 6. Plot
fig = px.bar(
    activity_counts,
    x='Number of Attacks',
    y='Activity Group',
    color='Activity Type',
    color_discrete_map=custom_colors,
    orientation='h',
    title='Number of Shark Attacks by Activity',
    text='Number of Attacks'
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Number of Attacks',
    yaxis_title='Activity Group',
    showlegend=False,
    margin=dict(t=80, b=40, l=1, r=1)
)
fig.show()



- Swimming and surfing are the two most common activities involved in shark attacks, both exceeding 1600 incidents.

- Fishing and diving also see high attack rates compared to other activity groups.

- Catastrophic and boating incidents have the lowest shark attack counts, under 200 and 400, respectively.

#### 2. Seasons corresponding to shark attacks

In [90]:
from utils import extract_month_or_nan, assign_season, get_season

# Ensure 'Season' is assigned based on 'Date'
df = assign_season(df)

# Then do your season counts and plotting
season_counts = df['Season'].value_counts().reset_index()
season_counts.columns = ['Season', 'Number of Attacks']

fig = px.bar(
    season_counts,
    x='Number of Attacks',
    y='Season',
    orientation='h',
    text='Number of Attacks',
    title='Number of Shark Attacks by Season'
)

fig.update_traces(marker_color='royalblue', textposition='outside')

fig.update_layout(
    xaxis_title='Number of Attacks',
    yaxis_title='Season',
    showlegend=False,
    margin=dict(t=100, b=80, l=1, r=1)
)

fig.show()


- Shark attack frequency peaks in summer, with over 2000 attacks, making it the riskiest season overall.

- Spring has the fewest attacks, followed by winter and autumn, showing a clear seasonal effect.

- However the seasons correspond to Northern Hemispheres, Souther Hemisphere patterns may differ.

### Bivariate Analysis

#### 1. Relationship between activity group and victim fatality

In [91]:
from utils import clean_fatal_column
df = clean_fatal_column(df)

# Calculate proportions
activity_fatal = (
    df.groupby("Activity_group")["Fatal Y/N"]
    .value_counts(normalize=True)
    .rename("Proportion")
    .reset_index()
)

# Extract only fatal proportions for sorting
fatal_order = (
    activity_fatal[activity_fatal["Fatal Y/N"] == "Did not survive"]
    .sort_values("Proportion", ascending=False)["Activity_group"]
)

# Create the horizontal stacked bar chart, sorted by fatality proportion
fig = px.bar(
    activity_fatal,
    y="Activity_group", 
    x="Proportion",
    color="Fatal Y/N",
    barmode="stack",
    category_orders={"Activity_group": fatal_order.tolist()},
    text=activity_fatal["Proportion"].apply(lambda x: f"{x:.0%}"),
    title="Proportion of Fatal vs Survived Incidents by Activity Group",
    labels={
        "Activity_group": "Activity Group",
        "Proportion": "Proportion",
        "Fatal Y/N": "Outcome",
    },
    orientation='h'
)

# Layout adjustments
fig.update_traces(textposition="inside")
fig.update_layout(
    yaxis_tickangle=0, 
    xaxis_tickformat=".0%",
    legend_title="Fatality Outcome",
    bargap=0.25,
)

fig.show()

- About 65% of the victims of **Catastrophes** like aircraft accidents over seas, drowning, Tsunamis, Floods and Hurricanes did not survive.
- **Boating** is activity that resulted in the second most fatality.
- **Surfing** is the activity that resulted in the least amount of fatal incidents.

In [92]:
from utils import clean_fatal_column
df = clean_fatal_column(df)

# Calculate counts instead of proportions for absolute numbers
activity_fatal_counts = (
    df.groupby(["Activity_group", "Fatal Y/N"])["Fatal Y/N"]
    .count()
    .rename("Count")
    .reset_index()
)

# Calculate fatality count per activity for ordering
fatal_counts = df[df["Fatal Y/N"] == "Did not survive"].groupby("Activity_group")["Fatal Y/N"].count()
fatal_order = fatal_counts.sort_values(ascending=False).index.tolist()

# Create horizontal stacked bar chart with absolute counts and labels
fig = px.bar(
    activity_fatal_counts,
    y="Activity_group",
    x="Count",
    color="Fatal Y/N",
    barmode="stack",
    category_orders={"Activity_group": fatal_order},
    text="Count",
    title="Number of Fatal vs Survived Incidents by Activity Group",
    labels={
        "Activity_group": " ",
        "Count": "Number of Incidents",
        "Fatal Y/N": "Outcome",
    },
    orientation="h",
)

fig.update_traces(textposition="inside")

fig.update_layout(
    height=600,  # or larger for more groups
    xaxis_title='Number of Incidents',
    showlegend=True,
    margin=dict(t=100, b=40, l=150, r=40)  # increase left margin for longer names
)
fig.update_yaxes(tickfont=dict(size=10))

fig.show()

#### 2. Relationship between Activity group and Season of the attack

In [93]:

#calculate total attack counts per season 
season_order_df = df.groupby('Season').size().reset_index(name='Total_Attacks') 
season_order = season_order_df.sort_values(by='Total_Attacks', ascending=False)['Season'].tolist() 
heatmap_data = df.groupby(['Season', 'Activity_group']).size().reset_index(name='Count')

fig = px.density_heatmap(
    heatmap_data,
    x='Season',
    y='Activity_group',
    z='Count',
    category_orders={'Season': season_order},
    color_continuous_scale='Blues',
    title='Summer season tends to be the high season for swimming shark attacks, whereas autumn is the peak in surfing attacks',
    labels={'Activity_group': ' ', 'Season': ' ', 'Count': 'Shark Attacks'}
)

# Add annotations
for i, row in heatmap_data.iterrows():
    fig.add_annotation(
        x=row['Season'],
        y=row['Activity_group'],
        text=str(int(row['Count'])),
        showarrow=False,
        font=dict(color='black', size=10)
    )

fig.show()

- Swimming and surfing activities experience the highest number of attacks in summer, exceeding 500 and 400 incidents respectively.

- Fishing-related attacks remain consistently high year-round but also maximize in summer.

- Wading and unknown activities show a notable drop outside the summer months.

- Catastrophe and boating activities are less affected by seasonal change, with relatively low and stable attack numbers across all seasons.

## Analysing if weather is a factor for some locations having the most attacks

In [94]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


### Extracting top 5 states in terms of attack for the period 2019-2024

In [95]:
from utils import clean_location_and_state

df = clean_location_and_state(df)

# Subset the dataset to attacks from 2019 onwards
recent_attacks = df[(df['Year'] >= 2019) & (df['Year'] <= 2024)]

# Group by 'state_clean' and count attacks
top_states_recent = recent_attacks.groupby('state_clean').size().reset_index(name='Count')

# Sort descending by count and select top 5 states
top_5_states = top_states_recent.sort_values(by='Count', ascending=False).head(5)

print("Top 5 states by attacks in recent 5 years:")
print(top_5_states)

Top 5 states by attacks in recent 5 years:
           state_clean  Count
30             Florida    133
136  Western Australia     44
44              Hawaii     44
98          Queensland     32
75     New South Wales     30


### Fetching weather data for the locations

In [96]:
from meteostat import Stations, Monthly
import pandas as pd
from datetime import datetime

from utils import clean_location_and_state

df = clean_location_and_state(df)

#mapping states to approximate coordinates
state_coords = {
    'Florida': (27.994402, -81.760254),
    'Western Australia': (-31.9505, 115.8605), #coordinates for Western Australia, used Perth approximately
    'Hawaii': (21.3069, -157.8583),
    'Queensland': (-20.9176, 142.7028),
    'New South Wales': (-31.2532, 146.9211),   
}

start = datetime(2019, 1, 1)
end = datetime(2024, 12, 31)

def fetch_yearly_avg_temp(name, lat, lon):
    stations = Stations().nearby(lat, lon).fetch(1)
    if stations.empty:
        print(f"No station found for {name}")
        return None
    station_id = stations.index[0]
    data = Monthly(station_id, start, end).fetch()
    if 'tavg' not in data.columns:
        if 'tmin' in data.columns and 'tmax' in data.columns:
            data['tavg'] = (data['tmin'] + data['tmax']) / 2
        else:
            print(f"Average temperature data unavailable for {name}")
            return None
    data = data.reset_index()
    data['Year'] = data['time'].dt.year
    yearly_avg = data.groupby('Year')['tavg'].mean().reset_index()
    yearly_avg.rename(columns={'tavg': 'Avg_Temperature_C'}, inplace=True)
    yearly_avg['state_clean'] = name
    return yearly_avg

#fetching temperature data only for top 5 states found
top_states_list = top_5_states['state_clean'].tolist()

temp_dfs = []
for state in top_states_list:
    coords = state_coords.get(state)
    if coords:
        temp_data = fetch_yearly_avg_temp(state, coords[0], coords[1])
        if temp_data is not None:
            temp_dfs.append(temp_data)
    else:
        print(f"Coordinates for {state} not found. Temperature data not fetched.")

#to combine all temperature data into one DataFrame
if temp_dfs:
    temp_all = pd.concat(temp_dfs, ignore_index=True)
    print("Temperature data fetched for top states:")
    print(temp_all.head(30))
else:
    print("No temperature data was fetched.")

Temperature data fetched for top states:
    Year  Avg_Temperature_C        state_clean
0   2019               19.1            Florida
1   2020          23.333333            Florida
2   2021             22.825            Florida
3   2022          22.766667            Florida
4   2023               23.4            Florida
5   2024          22.809091            Florida
6   2019          18.683333  Western Australia
7   2020               18.9  Western Australia
8   2021          18.641667  Western Australia
9   2022             18.675  Western Australia
10  2023          18.908333  Western Australia
11  2024              19.35  Western Australia
12  2019          26.258333             Hawaii
13  2020          26.058333             Hawaii
14  2021          25.741667             Hawaii
15  2022              25.75             Hawaii
16  2023               25.9             Hawaii
17  2024             25.675             Hawaii
18  2019               24.8         Queensland
19  2020          2

In [97]:
#aggregating shark attack data by year and state for number of attacks
attack_counts = recent_attacks.groupby(['Year', 'state_clean']).size().reset_index(name='Attack_Count')

#merge on year and state_clean
merged_df = pd.merge(
    attack_counts,
    temp_all,
    on=['Year', 'state_clean'],
    how='left'
)

print("Merged dataset sample:")
print(merged_df.head(10))

Merged dataset sample:
     Year            state_clean  Attack_Count  Avg_Temperature_C
0  2019.0                  Abaco             2               <NA>
1  2019.0                   Baja             1               <NA>
2  2019.0                  Bélep             1               <NA>
3  2019.0             California             6               <NA>
4  2019.0                 Cayman             1               <NA>
5  2019.0       Central Province             1               <NA>
6  2019.0                Chatham             1               <NA>
7  2019.0  Eastern Cape Province             4               <NA>
8  2019.0    Fernando de Noronha             2               <NA>
9  2019.0                Florida            26               19.1


In [98]:
selected_states = ["Florida", "Western Australia", "Hawaii", "Queensland", "New South Wales"]
filtered_df = merged_df[merged_df['state_clean'].isin(selected_states)]

#### 3. Relationship between the average temperature and attack trend

In [99]:
# Filter only top 5 states and years 2019–2024
top_states = ["Florida", "Western Australia", "Hawaii", "Queensland", "New South Wales"]
filtered_df = merged_df[
    (merged_df['state_clean'].isin(top_states)) &
    (merged_df['Year'].between(2019, 2024))
]

# Assign distinct colours for each state
colors = {
    "Florida": "#1f77b4",          # blue
    "Western Australia": "#ff7f0e", # orange
    "Hawaii": "#2ca02c",           # green
    "Queensland": "#d62728",       # red
    "New South Wales": "#9467bd"   # purple
}

fig = go.Figure()

for state in top_states:
    state_data = filtered_df[filtered_df['state_clean'] == state]
    color = colors[state]

    # Solid line for attacks
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Attack_Count"],
        name=f"{state} - Attacks",
        mode="lines+markers",
        line=dict(color=color, width=2),
        marker=dict(symbol="circle")
    ))

    # Dotted line for temperature (same colour)
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Avg_Temperature_C"],
        name=f"{state} - Temperature (°C)",
        mode="lines+markers",
        line=dict(color=color, dash="dot", width=2),
        marker=dict(symbol="x")
    ))

fig.update_layout(
    title="Year-wise Shark Attacks and Average Temperature (2019–2024)",
    xaxis_title="Year",
    yaxis_title="Count / Avg Temperature (°C)",
    template="plotly_white",
    legend_title="Legend",
    width=950,
    height=550
)

fig.show()

- Florida consistently has the highest annual shark attack counts among all states, especially in 2021.

- Western Australia and New South Wales have lower average temperatures compared to the other locations

- Attack counts in all states show annual variation but no clear correlation with temperature trends.

- Average water temperatures for states stay relatively stable year to year, not mirroring the fluctuations in attack counts.

##### Average temperature and attack trend - location wise split

In [100]:

top_states = ["Florida", "Western Australia", "Hawaii", "Queensland", "New South Wales"]

for state in top_states:
    state_data = merged_df[merged_df['state_clean'] == state]

    fig = go.Figure()

    # Line for attack count
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Attack_Count"],
        name="Attack Count",
        mode="lines+markers"
    ))

    # Line for average temperature
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Avg_Temperature_C"],
        name="Avg Temperature (°C)",
        mode="lines+markers",
        line=dict(dash="dot")
    ))

    fig.update_layout(
        title=f"Year-wise Shark Attacks and Avg Temperature in {state} (2019–2024)",
        xaxis_title="Year",
        yaxis_title="Value",
        legend_title="Variable",
        template="plotly_white"
    )

    fig.show()

#### 4. Relationship between location and fatality of the attack

In [101]:
from utils import clean_fatal_column, clean_location_and_state

df = clean_fatal_column(df)
df = clean_location_and_state(df)

# Step 1: Aggregate fatal vs non-fatal counts by location
fatality_location = (
    df.groupby(['location_clean', 'Fatal Y/N'])
      .size()
      .reset_index(name='Count')
)

# Step 2: Focus on top 10 locations by total number of attacks
top_locations = (
    df['location_clean']
    .value_counts()
    .head(10)
    .index
)
fatality_location = fatality_location[fatality_location['location_clean'].isin(top_locations)]

# Step 3: Horizontal bar chart
fig = px.bar(
    fatality_location,
    y='location_clean',     # location on Y-axis
    x='Count',              # numeric values on X-axis
    color='Fatal Y/N',
    barmode='group',
    orientation='h',        # makes it horizontal
    text='Count',           # adds value labels
    title='Fatal vs Non-Fatal Shark Attacks by Location',
    labels={'location_clean': 'Location', 'Count': 'Number of Attacks', 'Fatal Y/N': 'Outcome'},
    color_discrete_map={'Did not survive': '#d62728', 'Survived': '#2ca02c'}
)

# Step 4: Styling
fig.update_traces(textposition='outside')

fig.update_layout(
    plot_bgcolor='white',
    title_x=0.5,
    legend_title_text='Outcome',
    margin=dict(l=150)  # extra space for long location names
)

fig.show()


- "Unknown" locations have the highest actual count of attacks, possibly due to incomplete reporting.

- New Smyrna Beach and several Florida locations show high non-fatal attack counts, confirming their reputation as shark-prone areas.

- Most locations in the top 10 report a significantly higher number of non-fatal incidents than fatal ones.